In [1]:
import cutlass
import cutlass.cute as cute

import tract
from tract.categories import Nest_morphism, NestedTuple, Tuple_morphism, Tuple
from tract.test_utils import *
from tract.layout_utils import *

## Constructing tuples and nested tuples

In [2]:
S = (2,2,2)
T = ((2,2),(5,5))
U = ((2,2),4,(9,(3,3)))
print("S =", S)
print("T =", T)
print("U =", U)

S = (2, 2, 2)
T = ((2, 2), (5, 5))
U = ((2, 2), 4, (9, (3, 3)))


To create tuples of length 1, we must include a trailing comma:

In [3]:
S = (10,)
T = (10)
print("S =", S)
print("T =", T)

S = (10,)
T = 10


## Constructing layouts and morphisms

We construct layouts $L = S : D$ in `cute` as `L = cute.make_layout(shape=S, stride=D)`, as seen in these examples.

In [4]:
@cute.jit
def construct_example_layouts():
    A = cute.make_layout(shape=((4,4),4), stride=((16,1),4))
    B = cute.make_layout(shape=(8,64), stride=(64,1))
    C = cute.make_layout(shape=100, stride=2)
    print("A =", A)
    print("B =", B)
    print("C =", C)
construct_example_layouts()

A = ((4,4),4):((16,1),4)
B = (8,64):(64,1)
C = 100:2


We construct a nested tuple morphism 
$$
f: S \overset{\alpha}{\to} T
$$
in `tract` by specifying `f = tract.make_morphism(domain=S, codomain=T, map=alpha)`

In [5]:
f = Nest_morphism(domain=(4,4), codomain=(4,2,4), map=(1,3))
g = Nest_morphism(domain=(2,2,2,2), codomain=(2,2,2,2), map=(1,0,4,2))
h = Nest_morphism(domain=(16,(4,4),(4,4)), codomain=(16,4,4), map=(1,2,0,3,0))
print(f"{'f':<4}= {f}")
print(f"{'g':<4}= {g}")
print(f"{'h':<4}= {h}")

f   = (4,4) --(1, 3)--> (4,2,4)
g   = (2,2,2,2) --(1, 0, 4, 2)--> (2,2,2,2)
h   = (16,(4,4),(4,4)) --(1, 2, 0, 3, 0)--> (16,4,4)


## Translating between tractable layouts and morphisms

A layout $L$ is **tractable** if... We can check if $L$ is tractable with `tract.is_tractable(L)`. 

In [6]:
@cute.jit
def test_is_tractable():
    A = cute.make_layout(shape=(2,2,2), stride=(1,2,4))
    B = cute.make_layout(shape=(2,2,2), stride=(1,7,4))
    A_is_tractable = tract.is_tractable(A)
    B_is_tractable = tract.is_tractable(B)
    print(f"A =", A)
    print(f"A is tractable: {A_is_tractable}")
    print(f"B =", B)
    print(f"B is tractable: {B_is_tractable}")
test_is_tractable()

A = (2,2,2):(1,2,4)
A is tractable: True
B = (2,2,2):(1,7,4)
B is tractable: False


If $L$ is a tractable layout, then we can construct the standard representation $f_L$ with `tract.compute_morphism(L)`:

In [7]:
@cute.jit
def construct_standard_representation():
    L = cute.make_layout(shape=(2,2,2), stride=(1,2,4))
    f_L = tract.compute_morphism(L)
    print(f"{'L':<5}= {L}")
    print(f"{'f_L':<5}= {f_L}")
construct_standard_representation()

L    = (2,2,2):(1,2,4)
f_L  = (2,2,2) --(1, 2, 3)--> (2,2,2)


If $f$ is a tuple morphism, we can construct the layout $L_f$ encoded by $f$ with `tract.compute_layout(f)`.

In [8]:
@cute.jit
def compute_layout_example():
    f = Nest_morphism(domain=((5,5),8), codomain=(5,8,5), map=(1,3,2))
    L_f = tract.compute_layout(f)
    print(f"{'f':<5}= {f}")
    print(f"{'L_f':<5}= {L_f}")
compute_layout_example()

f    = ((5,5),8) --(1, 3, 2)--> (5,8,5)
L_f  = ((5,5),8):((1,40),5)


## Composition

When defined, this operation produces a layout $B \circ A$ from a pair of layouts $A$ and $B$. We can compute the composition $B \circ A$ in `cute` as `cute.composition(B, A)`:

In [9]:
@cute.jit
def composition_example():
    A = cute.make_layout(shape=((4,4),4), stride=((16,1),4))
    B = cute.make_layout(shape=(8,64), stride=(64,1))
    B_o_A = cute.composition(B,A)
    print(f"{'A':<5}= {A}")
    print(f"{'B':<5}= {B}")
    print(f"{'B∘A':<5}= {B_o_A}")
composition_example()

A    = ((4,4),4):((16,1),4)
B    = (8,64):(64,1)
B∘A  = ((4,4),(2,2)):((2,64),(256,1))


If $f$ and $g$ are composable nested tuple morphisms, we can compute the composition $g \circ f$ in `tract` as `f.compose(g)`:

In [10]:
f = Nest_morphism(domain=((2,2),(2,2)), codomain=((2,2,2),(2,2,2)), map=(3,2,6,5))
g = Nest_morphism(domain=((2,2,2),(2,2,2)), codomain=(2,2,2,2), map=(1,0,2,0,3,4))
g_o_f = tract.compose(f, g)
print(f"{'f':<5}= {f}")
print(f"{'g':<5}= {g}")
print(f"{'g∘f':<5}= {g_o_f}")

f    = ((2,2),(2,2)) --(3, 2, 6, 5)--> ((2,2,2),(2,2,2))
g    = ((2,2,2),(2,2,2)) --(1, 0, 2, 0, 3, 4)--> (2,2,2,2)
g∘f  = ((2,2),(2,2)) --(2, 0, 4, 3)--> (2,2,2,2)


## Coalesce

This operation produces a layout $\text{coal}(A)$ from $A$ that is in a precise sense of *minimal complexity*. We can compute $\text{coal}(A)$ in `cute` with `cute.coalesce(A)`.

In [11]:
@cute.jit 
def coalesce_example():
     A = cute.make_layout(shape = ((2,2),(2,2),(5,5)), stride = ((1,2),(16,32),(64,640)))
     coal_A = cute.coalesce(A)
     print(f"{'A':<7}= {A}")
     print(f"{'coal_A':<7}= {coal_A}")
coalesce_example()

A      = ((2,2),(2,2),(5,5)):((1,2),(16,32),(64,640))
coal_A = (4,20,5):(1,16,640)


There is also a **relative coalesce** operation (sometimes called **by-mode coalesce**) $A \mapsto \text{coal}(A, S)$, which receives as input an additional nested tuple $S$ which is *refined* by the shape of $A$. We can compute $\text{coal}(A, S)$ in `cute` with `cute.coalesce(A, target_profile=S)`. 

In [12]:
@cute.jit
def relative_coalesce_example():
     A = cute.make_layout(shape = ((2,2),(3,3),(5,5)), stride = ((1,2),(4,12),(36,180)))
     S = ((2,2),9,25)
     coal_A_over_S = cute.coalesce(A,target_profile=S)
     print(f"{'A':<15}= {A}")
     print(f"{'S':<15}= {S}")
     print(f"{'coal_A_over_S':<15}= {coal_A_over_S}")
relative_coalesce_example()

A              = ((2,2),(3,3),(5,5)):((1,2),(4,12),(36,180))
S              = ((2, 2), 9, 25)
coal_A_over_S  = ((2,2),9,25):((1,2),4,36)


If $f$ is a nested tuple morphism, we may form $\text{coal}(f)$. We compute $\text{coal}(f)$ in `tract` as `tract.coalesce(f)`. 

In [13]:
f = Nest_morphism(domain=(2,2,10,10), codomain=(2,2,2,10,10), map=(1,2,4,5))
coal_f = f.coalesce()
print(f"{'f':<7}= {f}")
print(f"{'coal_f':<7}= {coal_f}")

f      = (2,2,10,10) --(1, 2, 4, 5)--> (2,2,2,10,10)
coal_f = (4,100) --(1, 3)--> (4,2,100)


## Complement

When defined, this operation produces a layout $\text{comp}(A, N)$ from a layout $A$ and a positive integer $N$. We can compute $\text{comp}(A, N)$ in `cute` with `cute.complement(A, N)`:

In [14]:
@cute.jit
def complement_example():
     A = cute.make_layout(shape = ((2,2),(2,2)), stride = ((8,2),(64,256)))
     comp_A = cute.complement(A,4096)
     print(f"{'A':<7}= {A}")
     print(f"{'comp_A':<7}= {comp_A}")
complement_example()

A      = ((2,2),(2,2)):((8,2),(64,256))
comp_A = (2,2,4,2,8):(1,4,16,128,512)


If $f$ is a nested tuple morphism, we may form the complement $f^c$ of $f$. We compute $f^c$ in `tract` as `tract.complement(f)`. 

In [15]:
f = Nest_morphism(domain=(2,2), codomain=(2,5,2,5), map=(1,3))
comp_f = f.complement()
print(f"{'f':<7}= {f}")
print(f"{'comp_f':<7}= {comp_f}")

f      = (2,2) --(1, 3)--> (2,5,2,5)
comp_f = (5,5) --(2, 4)--> (2,5,2,5)


## Logical Division

When defined, this operation produces a layout $A \oslash B$ from a pair of layouts $A$ and $B$. We compute $A \oslash B$ in `cute` with `cute.logical_divide(A, B)`.

In [16]:
@cute.jit
def logical_divide_example():
    A = cute.make_layout((64,32), stride = (32,1))
    B = cute.make_layout((4,4), stride = (1,64))
    quotient = cute.logical_divide(A,B)
    print(f"{'A':<9}= {A}")
    print(f"{'B':<9}= {B}")
    print(f"{'quotient':<9}= {quotient}")
logical_divide_example()

A        = (64,32):(32,1)
B        = (4,4):(1,64)
quotient = ((4,4),(16,8)):((32,1),(128,4))


If $f$ and $g$ are nested tuple morphisms and $g$ divides $f$, then we may form the logical division $f \oslash g$. We compute $f \oslash g$ in `tract` with `tract.logical_divide(f, g)`. 

In [17]:
f = tract.make_morphism(domain=(4, 8, 4, 8), codomain=(4, 8, 4, 8), map=(1, 2, 3, 4))
g = tract.make_morphism(domain=(4, 4), codomain=(4, 8, 4, 8), map=(1, 3))
quotient = tract.logical_divide(f, g)
print(f"{'f':<10}= {f}")
print(f"{'g':<10}= {g}")
print(f"{'quotient':<10}= {quotient}")

f         = (4,8,4,8) --(1, 2, 3, 4)--> (4,8,4,8)
g         = (4,4) --(1, 3)--> (4,8,4,8)
quotient  = ((4,4),(8,8)) --(1, 3, 2, 4)--> (4,8,4,8)


## Logical Product

When defined, this operation produces a layout $A \otimes B$ from a pair of layouts $A$ and $B$. We compute $A \otimes B$ in `cute` as `cute.logical_product(A, B)`. 

In [18]:
@cute.jit
def logical_product_example():
    A = cute.make_layout((3,10,10), stride = (200,1,20))
    B = cute.make_layout((2,2), stride = (1,2))
    product = cute.logical_product(A,B)
    print(f"{'A':<9}= {A}")
    print(f"{'B':<9}= {B}")
    print(f"{'product':<9}= {product}")
logical_product_example()

A        = (3,10,10):(200,1,20)
B        = (2,2):(1,2)
product  = ((3,10,10),(2,2)):((200,1,20),(10,600))


If $f$ and $g$ are nested tuple morphisms that are product-admissible, we may form the logical product $f \otimes g$. We compute $f \otimes g$ in `tract` with `tract.logical_product(f, g)`. 

In [19]:
f = tract.make_morphism(domain=(2, 2), codomain=(2, 2, 5, 5), map=(1, 2))
g = tract.make_morphism(domain=(5,5), codomain=(5,5), map=(2,1))
product = tract.logical_product(f, g)
print(f"{'f':<10}= {f}")
print(f"{'g':<10}= {g}")
print(f"{'product':<10}= {product}")

f         = (2,2) --(1, 2)--> (2,2,5,5)
g         = (5,5) --(2, 1)--> (5,5)
product   = ((2,2),(5,5)) --(1, 2, 4, 3)--> (2,2,5,5)
